In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
batch_size = 64
learning_rate = 1e-2
num_epochs = 5 # 训练次数
# 判断GPU是否可用
use_gpu = torch.cuda.is_available()

# 下载训练集 MNIST 手写数字训练集
# 数据是datasets类型的
train_dataset = datasets.FashionMNIST(
    root='./datasets', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='./datasets', train=False, transform=transforms.ToTensor())
#　将数据处理成 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # 选择打乱数据
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # 选择不打乱数据

26422272it [00:26, 980189.71it/s]                               


Extracting ./datasets/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./datasets/FashionMNIST/raw



29696it [00:00, 133589.51it/s]                          


Extracting ./datasets/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/FashionMNIST/raw



4422656it [00:04, 983847.41it/s]                              


Extracting ./datasets/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/FashionMNIST/raw



6144it [00:00, 5688698.41it/s]          

Extracting ./datasets/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/FashionMNIST/raw



In [5]:
class neuralNetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(neuralNetwork, self).__init__() # super() 函数是用于调用父类(超类)的一个方法
# Sequential()表示将一个有序的模块写在一起，也就相当于将神经网络的层按顺序放在一起，这样可以方便结构显示
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True)) # 表示使用ReLU激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

# 定义向前传播
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [6]:
model = neuralNetwork(28 * 28, 300, 100, 10)

In [7]:
criterion = nn.CrossEntropyLoss() # 定义损失函数类型，使用交叉熵
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # 定义优化器，使用随机梯度下降

In [8]:
# 开始模型训练
for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    running_loss = 0.0 # 初始值
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        img, label = data
        img = img.view(img.size(0), -1) # 将图片展开为28*28
        # 使用GPU？
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # 向前传播
        out = model(img) # 前向传播
        loss = criterion(out, label) # 计算loss
        running_loss += loss.item() # loss求和
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad() # 梯度归零
        loss.backward() # 后向传播
        optimizer.step() # 更新参数

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')

**********
epoch 1
[1/5] Loss: 2.194906, Acc: 0.374375
[1/5] Loss: 1.982077, Acc: 0.439922
[1/5] Loss: 1.790408, Acc: 0.477674
Finish 1 epoch, Loss: 1.771038, Acc: 0.480877
**********
epoch 2
[2/5] Loss: 1.256835, Acc: 0.579167
[2/5] Loss: 1.220822, Acc: 0.598385
[2/5] Loss: 1.204185, Acc: 0.611285
Finish 2 epoch, Loss: 1.202303, Acc: 0.612740
**********
epoch 3
[3/5] Loss: 1.117963, Acc: 0.659948
[3/5] Loss: 1.102837, Acc: 0.663854
[3/5] Loss: 1.097137, Acc: 0.665243
Finish 3 epoch, Loss: 1.095320, Acc: 0.665911
**********
epoch 4
[4/5] Loss: 1.062760, Acc: 0.674219
[4/5] Loss: 1.052471, Acc: 0.677734
[4/5] Loss: 1.043747, Acc: 0.681059
Finish 4 epoch, Loss: 1.041778, Acc: 0.681753
**********
epoch 5
[5/5] Loss: 0.981535, Acc: 0.687760
[5/5] Loss: 0.976716, Acc: 0.688932
[5/5] Loss: 0.967162, Acc: 0.692743
Finish 5 epoch, Loss: 0.965909, Acc: 0.693064
